In [1]:
import pandas as pd
import numpy as np
import ast

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\fabio\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#annotated_df = pd.read_csv("data/annotated_dataset/annotated_texts_repr.csv", sep=",", encoding="utf-8")
annotated_df = pd.read_csv("data/annotated_dataset/annotated_texts_repr_pro_complete.csv", sep=",", encoding="utf-8")
meta_df = pd.read_csv("data/cleaned_dataset/meta.tsv", sep=",", encoding="utf-8")
merged_df = annotated_df.merge(meta_df, how='left', left_on='id', right_on="ID")

ling_prof_df = merged_df[["id", "pop_sum","polarization","Speaker_ID", "Speaker_party","Party_orientation","linguistic_profile_pro", "tfidf_pro", "doc_embedding_pro"]]
diz = {
    "LN-Aut": "Lega",
    "L-SP": "Lega",
    "M5S.1": "M5S",
    "M5S.2": "M5S",
}
ling_prof_df = ling_prof_df.replace({"Speaker_party": diz})
ling_prof_df = ling_prof_df.rename(columns={"linguistic_profile_pro": "linguistic_profile", "tfidf_pro": "tfidf", "doc_embedding_pro":"doc_embedding"})

In [3]:
def mean_std_lists(lists):
    lists = np.array([ast.literal_eval(item) for item in lists])
    return np.mean(lists, axis=0).tolist(), np.std(lists, axis=0).tolist()

#Aggregazione per senatore
df_grouped = ling_prof_df.groupby('Speaker_ID', as_index=False).agg({
    'pop_sum': lambda x: x.sum() / x.count(),
    'polarization': lambda x: x.sum() / x.count(),
    'Speaker_party': lambda x: x.mode()[0],
    'Party_orientation': lambda x: x.mode()[0]
})

#Media e std delle rappresentazioni del testo (per tutti i testi di un senatore)
for col in ['linguistic_profile', 'tfidf', 'doc_embedding']:
    df_grouped[col], df_grouped[col + '_std'] = zip(*ling_prof_df.groupby('Speaker_ID')[col].apply(lambda x: mean_std_lists(list(x))))

df_grouped["linguistic_profile"] = df_grouped["linguistic_profile"] + df_grouped["linguistic_profile_std"]
df_grouped["tfidf"] = df_grouped["tfidf"] + df_grouped["tfidf_std"]
df_grouped["doc_embedding"] = df_grouped["doc_embedding"] + df_grouped["doc_embedding_std"]
    
group_sizes = ling_prof_df.groupby('Speaker_ID').size().reset_index(name='Count_Per_Group')
df_grouped = df_grouped.merge(group_sizes, on='Speaker_ID')

In [5]:
#df_grouped.to_csv("data/annotated_dataset/speaker_data.csv", index=False)

In [7]:
scaler = StandardScaler()
def linearSvcBestParams(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    svc = LinearSVC(dual=False, max_iter=10000)

    param_grid = {
        'C': [0.01, 0.1, 1],
        'loss': ['squared_hinge', 'hinge'],
        'penalty': ['l1', 'l2'],
        'class_weight': ['balanced']
    }

    grid_search = GridSearchCV(svc, param_grid, cv=2, scoring='accuracy', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)
    print("Best Cross-Validation Accuracy:", grid_search.best_score_)

    best_model = LinearSVC(dual=False, **best_params)
    
    X_scaled = scaler.transform(X)
    predictions = cross_val_predict(best_model, X_scaled, y, cv=3)
    
    print(classification_report(y, predictions))

#### Estraggo X e y

In [5]:
X_ling = np.vstack(df_grouped["linguistic_profile"].values)
X_tfidf = np.vstack(df_grouped["tfidf"].values)
X_embed = np.vstack(df_grouped["doc_embedding"].values)

X_list = [X_ling,X_tfidf,X_embed]

y_party = np.vstack(df_grouped["Speaker_party"].values)
y_ori = np.vstack(df_grouped["Party_orientation"].values)

## Classificazione

In [6]:
clf_linear_svc = LinearSVC(C=1.0, random_state=42)

In [11]:
def classificazione(X_list, y):
    for X in X_list:
        predictions = cross_val_predict(clf_linear_svc, X, y, cv=3)
        print(classification_report(y, predictions))

In [56]:
classificazione(X_list, y_party)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

              precision    recall  f1-score   support

       FI-BP       0.11      0.02      0.04        48
         FdI       0.00      0.00      0.00        22
        Lega       0.14      0.24      0.17        67
         M5S       0.25      0.29      0.27       129
         NCD       0.00      0.00      0.00        25
          PD       0.32      0.36      0.34       148
         PdL       0.19      0.23      0.21        53
        SCpI       0.00      0.00      0.00        15

    accuracy                           0.24       507
   macro avg       0.13      0.14      0.13       507
weighted avg       0.21      0.24      0.22       507



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

       FI-BP       0.54      0.58      0.56        48
         FdI       0.47      0.41      0.44        22
        Lega       0.42      0.37      0.40        67
         M5S       0.61      0.62      0.61       129
         NCD       0.17      0.04      0.06        25
          PD       0.57      0.72      0.63       148
         PdL       0.44      0.34      0.38        53
        SCpI       0.08      0.07      0.07        15

    accuracy                           0.53       507
   macro avg       0.41      0.39      0.40       507
weighted avg       0.50      0.53      0.51       507



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

       FI-BP       0.47      0.44      0.45        48
         FdI       0.35      0.32      0.33        22
        Lega       0.31      0.30      0.31        67
         M5S       0.48      0.49      0.49       129
         NCD       0.25      0.08      0.12        25
          PD       0.51      0.66      0.58       148
         PdL       0.41      0.32      0.36        53
        SCpI       0.10      0.07      0.08        15

    accuracy                           0.45       507
   macro avg       0.36      0.33      0.34       507
weighted avg       0.43      0.45      0.44       507



In [57]:
classificazione(X_list, y_ori)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

                             precision    recall  f1-score   support

                     Centro       0.27      0.20      0.23        15
 Centro verso centro-destra       0.21      0.16      0.18        25
              Centro-destra       0.24      0.70      0.36       101
            Centro-sinistra       0.47      0.36      0.41       148
Destra verso estrema destra       0.33      0.18      0.23        89
                Pigliatutto       0.44      0.06      0.11       129

                   accuracy                           0.31       507
                  macro avg       0.33      0.28      0.25       507
               weighted avg       0.38      0.31      0.28       507



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

                     Centro       0.09      0.07      0.08        15
 Centro verso centro-destra       0.12      0.04      0.06        25
              Centro-destra       0.60      0.54      0.57       101
            Centro-sinistra       0.54      0.67      0.60       148
Destra verso estrema destra       0.49      0.49      0.49        89
                Pigliatutto       0.64      0.62      0.63       129

                   accuracy                           0.55       507
                  macro avg       0.42      0.41      0.41       507
               weighted avg       0.54      0.55      0.54       507



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

                     Centro       0.09      0.07      0.08        15
 Centro verso centro-destra       0.17      0.04      0.06        25
              Centro-destra       0.55      0.49      0.52       101
            Centro-sinistra       0.49      0.64      0.55       148
Destra verso estrema destra       0.40      0.46      0.43        89
                Pigliatutto       0.52      0.43      0.47       129

                   accuracy                           0.48       507
                  macro avg       0.37      0.35      0.35       507
               weighted avg       0.47      0.48      0.47       507



In [8]:
linearSvcBestParams(X_tfidf, y_party)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.701263229771253


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

       FI-BP       0.58      0.67      0.62        48
         FdI       0.79      0.86      0.83        22
        Lega       0.84      0.78      0.81        67
         M5S       0.85      0.90      0.87       129
         NCD       0.42      0.44      0.43        25
          PD       0.75      0.80      0.77       148
         PdL       0.81      0.57      0.67        53
        SCpI       0.12      0.07      0.09        15

    accuracy                           0.75       507
   macro avg       0.65      0.63      0.64       507
weighted avg       0.74      0.75      0.74       507



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
linearSvcBestParams(X_tfidf, y_ori)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.7456591718285128


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                             precision    recall  f1-score   support

                     Centro       0.21      0.20      0.21        15
 Centro verso centro-destra       0.30      0.48      0.37        25
              Centro-destra       0.77      0.71      0.74       101
            Centro-sinistra       0.82      0.79      0.81       148
Destra verso estrema destra       0.85      0.85      0.85        89
                Pigliatutto       0.84      0.84      0.84       129

                   accuracy                           0.77       507
                  macro avg       0.63      0.65      0.64       507
               weighted avg       0.78      0.77      0.77       507



### Selezione di soli due partiti

In [10]:
X_ling = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S"])]["linguistic_profile"].values)
X_tfidf = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S"])]["tfidf"].values)
X_embed = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S"])]["doc_embedding"].values)

X_list = [X_ling,X_tfidf,X_embed]

y_party = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S"])]["Speaker_party"].values)
y_ori = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S"])]["Party_orientation"].values)

In [59]:
classificazione(X_list, y_party)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

              precision    recall  f1-score   support

         M5S       0.59      0.88      0.70       129
          PD       0.81      0.47      0.59       148

    accuracy                           0.66       277
   macro avg       0.70      0.67      0.65       277
weighted avg       0.71      0.66      0.64       277

              precision    recall  f1-score   support

         M5S       0.72      0.74      0.73       129
          PD       0.77      0.74      0.76       148

    accuracy                           0.74       277
   macro avg       0.74      0.74      0.74       277
weighted avg       0.74      0.74      0.74       277

              precision    recall  f1-score   support

         M5S       0.68      0.70      0.69       129
          PD       0.73      0.72      0.72       148

    accuracy                           0.71       277
   macro avg       0.71      0.71      0.71       277
weighted avg       0.71      0.71      0.71       277



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

In [60]:
classificazione(X_list, y_ori)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

                 precision    recall  f1-score   support

Centro-sinistra       0.69      0.63      0.66       148
    Pigliatutto       0.61      0.67      0.64       129

       accuracy                           0.65       277
      macro avg       0.65      0.65      0.65       277
   weighted avg       0.65      0.65      0.65       277



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                 precision    recall  f1-score   support

Centro-sinistra       0.77      0.74      0.76       148
    Pigliatutto       0.72      0.74      0.73       129

       accuracy                           0.74       277
      macro avg       0.74      0.74      0.74       277
   weighted avg       0.74      0.74      0.74       277

                 precision    recall  f1-score   support

Centro-sinistra       0.73      0.72      0.72       148
    Pigliatutto       0.68      0.70      0.69       129

       accuracy                           0.71       277
      macro avg       0.71      0.71      0.71       277
   weighted avg       0.71      0.71      0.71       277



In [11]:
linearSvcBestParams(X_tfidf, y_party)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.8824733824733825
              precision    recall  f1-score   support

         M5S       0.88      0.91      0.89       129
          PD       0.92      0.89      0.90       148

    accuracy                           0.90       277
   macro avg       0.90      0.90      0.90       277
weighted avg       0.90      0.90      0.90       277



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

In [12]:
linearSvcBestParams(X_tfidf, y_ori)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.8824733824733825
                 precision    recall  f1-score   support

Centro-sinistra       0.92      0.89      0.90       148
    Pigliatutto       0.88      0.91      0.89       129

       accuracy                           0.90       277
      macro avg       0.90      0.90      0.90       277
   weighted avg       0.90      0.90      0.90       277



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

### Selezione di soli quattro partiti

In [13]:
X_ling = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega", "FdI"])]["linguistic_profile"].values)
X_tfidf = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","FdI"])]["tfidf"].values)
X_embed = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","FdI"])]["doc_embedding"].values)

X_list = [X_ling,X_tfidf,X_embed]

y_party = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","FdI"])]["Speaker_party"].values)
y_ori = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","FdI"])]["Party_orientation"].values)

In [62]:
classificazione(X_list, y_party)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

              precision    recall  f1-score   support

         FdI       0.33      0.05      0.08        22
        Lega       0.30      0.04      0.08        67
         M5S       0.63      0.24      0.35       129
          PD       0.45      0.93      0.61       148

    accuracy                           0.47       366
   macro avg       0.43      0.32      0.28       366
weighted avg       0.48      0.47      0.39       366



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

         FdI       0.64      0.32      0.42        22
        Lega       0.51      0.45      0.48        67
         M5S       0.64      0.67      0.65       129
          PD       0.71      0.76      0.74       148

    accuracy                           0.65       366
   macro avg       0.62      0.55      0.57       366
weighted avg       0.64      0.65      0.64       366



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

         FdI       0.45      0.23      0.30        22
        Lega       0.40      0.33      0.36        67
         M5S       0.54      0.58      0.56       129
          PD       0.65      0.72      0.68       148

    accuracy                           0.57       366
   macro avg       0.51      0.46      0.48       366
weighted avg       0.56      0.57      0.56       366



In [63]:
classificazione(X_list, y_ori)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

                             precision    recall  f1-score   support

            Centro-sinistra       0.62      0.52      0.57       148
Destra verso estrema destra       0.53      0.28      0.37        89
                Pigliatutto       0.47      0.71      0.57       129

                   accuracy                           0.53       366
                  macro avg       0.54      0.50      0.50       366
               weighted avg       0.55      0.53      0.52       366



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

            Centro-sinistra       0.69      0.74      0.71       148
Destra verso estrema destra       0.58      0.55      0.57        89
                Pigliatutto       0.66      0.62      0.64       129

                   accuracy                           0.65       366
                  macro avg       0.64      0.64      0.64       366
               weighted avg       0.65      0.65      0.65       366



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

            Centro-sinistra       0.64      0.70      0.67       148
Destra verso estrema destra       0.51      0.48      0.49        89
                Pigliatutto       0.57      0.53      0.55       129

                   accuracy                           0.58       366
                  macro avg       0.57      0.57      0.57       366
               weighted avg       0.58      0.58      0.58       366



In [14]:
linearSvcBestParams(X_tfidf, y_party)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8116438356164384
              precision    recall  f1-score   support

         FdI       0.87      0.91      0.89        22
        Lega       0.87      0.81      0.84        67
         M5S       0.87      0.94      0.90       129
          PD       0.92      0.88      0.90       148

    accuracy                           0.89       366
   macro avg       0.88      0.88      0.88       366
weighted avg       0.89      0.89      0.89       366



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
linearSvcBestParams(X_tfidf, y_ori)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.8527397260273972


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                             precision    recall  f1-score   support

            Centro-sinistra       0.92      0.86      0.89       148
Destra verso estrema destra       0.82      0.90      0.86        89
                Pigliatutto       0.85      0.85      0.85       129

                   accuracy                           0.87       366
                  macro avg       0.86      0.87      0.87       366
               weighted avg       0.87      0.87      0.87       366



### Selezione di partiti con support >47

In [16]:
X_ling = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega", "PdL", "FI-BP"])]["linguistic_profile"].values)
X_tfidf = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","PdL", "FI-BP"])]["tfidf"].values)
X_embed = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","PdL", "FI-BP"])]["doc_embedding"].values)

X_list = [X_ling,X_tfidf,X_embed]

y_party = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","PdL", "FI-BP"])]["Speaker_party"].values)
y_ori = np.vstack(df_grouped[df_grouped["Speaker_party"].isin(["PD", "M5S", "Lega","PdL", "FI-BP"])]["Party_orientation"].values)

In [26]:
classificazione(X_list, y_party)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

              precision    recall  f1-score   support

       FI-BP       0.23      0.06      0.10        48
        Lega       0.07      0.01      0.02        67
         M5S       0.43      0.16      0.23       129
          PD       0.40      0.57      0.47       148
         PdL       0.12      0.38      0.19        53

    accuracy                           0.29       445
   macro avg       0.25      0.24      0.20       445
weighted avg       0.31      0.29      0.26       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

       FI-BP       0.55      0.58      0.57        48
        Lega       0.47      0.36      0.41        67
         M5S       0.64      0.66      0.65       129
          PD       0.64      0.73      0.68       148
         PdL       0.50      0.38      0.43        53

    accuracy                           0.60       445
   macro avg       0.56      0.54      0.55       445
weighted avg       0.59      0.60      0.59       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

              precision    recall  f1-score   support

       FI-BP       0.50      0.54      0.52        48
        Lega       0.33      0.28      0.31        67
         M5S       0.56      0.51      0.53       129
          PD       0.57      0.68      0.62       148
         PdL       0.39      0.30      0.34        53

    accuracy                           0.51       445
   macro avg       0.47      0.46      0.46       445
weighted avg       0.50      0.51      0.50       445



In [27]:
classificazione(X_list, y_ori)

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_class

                             precision    recall  f1-score   support

              Centro-destra       0.25      0.56      0.35       101
            Centro-sinistra       0.46      0.53      0.49       148
Destra verso estrema destra       0.36      0.07      0.12        67
                Pigliatutto       0.50      0.13      0.21       129

                   accuracy                           0.35       445
                  macro avg       0.39      0.32      0.29       445
               weighted avg       0.41      0.35      0.32       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

              Centro-destra       0.63      0.61      0.62       101
            Centro-sinistra       0.63      0.72      0.67       148
Destra verso estrema destra       0.50      0.37      0.43        67
                Pigliatutto       0.66      0.65      0.65       129

                   accuracy                           0.62       445
                  macro avg       0.60      0.59      0.59       445
               weighted avg       0.62      0.62      0.62       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn

                             precision    recall  f1-score   support

              Centro-destra       0.57      0.54      0.56       101
            Centro-sinistra       0.57      0.66      0.61       148
Destra verso estrema destra       0.33      0.27      0.30        67
                Pigliatutto       0.53      0.50      0.52       129

                   accuracy                           0.53       445
                  macro avg       0.50      0.49      0.50       445
               weighted avg       0.52      0.53      0.52       445



In [17]:
linearSvcBestParams(X_tfidf, y_party)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.7219101123595506


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was pa

              precision    recall  f1-score   support

       FI-BP       0.62      0.75      0.68        48
        Lega       0.88      0.79      0.83        67
         M5S       0.86      0.82      0.84       129
          PD       0.85      0.85      0.85       148
         PdL       0.64      0.68      0.66        53

    accuracy                           0.80       445
   macro avg       0.77      0.78      0.77       445
weighted avg       0.81      0.80      0.80       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [18]:
linearSvcBestParams(X_tfidf, y_ori)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\svm\_classes.py", line 315, in fit
    self.coef_, self.inter

Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.800561797752809


C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                             precision    recall  f1-score   support

              Centro-destra       0.83      0.85      0.84       101
            Centro-sinistra       0.89      0.87      0.88       148
Destra verso estrema destra       0.83      0.87      0.85        67
                Pigliatutto       0.87      0.85      0.86       129

                   accuracy                           0.86       445
                  macro avg       0.85      0.86      0.86       445
               weighted avg       0.86      0.86      0.86       445



C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
